In [1]:
# --- Librerías ---
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [4]:
# Read the K2Processed.csv file into a DataFrame
k2_df = pd.read_csv('../../data/processed\k2Processed.csv')

# Normalize disposition labels
k2_df['disposition'] = k2_df['disposition'].replace({
    'CONFIRMED': 'CP',
    'FALSE POSITIVE': 'FP',
    'CANDIDATE': 'PC'
})

# Rename the disposition column to 'Status'
k2_df = k2_df.rename(columns={'disposition': 'Status'})

# Count the occurrences of each disposition type
disposition_counts = k2_df['Status'].value_counts()

# Display the counts for the requested dispositions
print("Counts for K2Processed.csv:")
print(disposition_counts[['CP', 'FP', 'PC']])

Counts for K2Processed.csv:
Status
CP    1019
FP      10
PC     192
Name: count, dtype: int64


<>:2: SyntaxWarning: invalid escape sequence '\k'
<>:2: SyntaxWarning: invalid escape sequence '\k'
C:\Users\anala\AppData\Local\Temp\ipykernel_26392\579437714.py:2: SyntaxWarning: invalid escape sequence '\k'
  k2_df = pd.read_csv('../../data/processed\k2Processed.csv')


In [5]:
# Read the KeplerProcessed.csv file into a DataFrame
kepler_df = pd.read_csv('../../data/processed\keplerProcessed.csv')

# Normalize disposition labels
kepler_df['koi_disposition'] = kepler_df['koi_disposition'].replace({
    'CONFIRMED': 'CP',
    'FALSE POSITIVE': 'FP',
    'CANDIDATE': 'PC'
})

# Rename the koi_disposition column to 'Status'
kepler_df = kepler_df.rename(columns={'koi_disposition': 'Status'})

# Count the occurrences of each value in the 'Status' column
koi_disposition_counts = kepler_df['Status'].value_counts()

# Display the counts
print("Counts for KeplerProcessed.csv:")
print(koi_disposition_counts)

Counts for KeplerProcessed.csv:
Status
FP    3735
CP    2731
PC    1356
Name: count, dtype: int64


<>:2: SyntaxWarning: invalid escape sequence '\k'
<>:2: SyntaxWarning: invalid escape sequence '\k'
C:\Users\anala\AppData\Local\Temp\ipykernel_26392\1219379012.py:2: SyntaxWarning: invalid escape sequence '\k'
  kepler_df = pd.read_csv('../../data/processed\keplerProcessed.csv')


In [6]:
# Read the TOIProcessed.csv file into a DataFrame
toi_df = pd.read_csv('../../data/processed\TOIProcessed.csv')

# Rename the tfopwg_disp column to 'Status'
toi_df = toi_df.rename(columns={'tfopwg_disp': 'Status'})

# Count the occurrences of each value in the 'Status' column
tfopwg_disp_counts = toi_df['Status'].value_counts()

# Display the counts
print("Counts for TOIProcessed.csv:")
print(tfopwg_disp_counts)

Counts for TOIProcessed.csv:
Status
PC     3008
FP      812
CP      605
KP      365
APC     299
FA       55
Name: count, dtype: int64


<>:2: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\T'
C:\Users\anala\AppData\Local\Temp\ipykernel_26392\1649327321.py:2: SyntaxWarning: invalid escape sequence '\T'
  toi_df = pd.read_csv('../../data/processed\TOIProcessed.csv')


## Concat

In [7]:
# Combine the three dataframes
combined_df = pd.concat([k2_df, kepler_df, toi_df], ignore_index=True)
# Display the first few rows and the info of the combined dataframe to verify
print("Combined DataFrame:")
display(combined_df.head())
print("\nInfo of the combined DataFrame:")
combined_df.info()

Combined DataFrame:


,pl_name,hostname,default_flag,Status,disp_refname,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,...,pl_trandeperr2,pl_insol,pl_eqt,st_tmag,st_tmagerr1,st_tmagerr2,st_dist,st_disterr1,st_disterr2,toi_created
0,BD+20 594 b,BD+20 594,0.0,CP,Espinoza et al. 2016,1.0,1.0,Transit,2016.0,K2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BD+20 594 b,BD+20 594,0.0,CP,Espinoza et al. 2016,1.0,1.0,Transit,2016.0,K2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BD+20 594 b,BD+20 594,1.0,CP,Espinoza et al. 2016,1.0,1.0,Transit,2016.0,K2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EPIC 201111557.01,EPIC 201111557,1.0,PC,Livingston et al. 2018,1.0,0.0,Transit,2018.0,K2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EPIC 201127519.01,EPIC 201127519,1.0,PC,Livingston et al. 2018,1.0,0.0,Transit,2018.0,K2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



Info of the combined DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14190 entries, 0 to 14189
Columns: 124 entries, pl_name to toi_created
dtypes: float64(105), object(19)
memory usage: 13.4+ MB


In [8]:
# Elimina el 50% de las filas que contienen valores nulos en kepler_df
num_rows_to_drop = int(kepler_df.isnull().any(axis=1).sum() * 0.5)
rows_with_nan = kepler_df[kepler_df.isnull().any(axis=1)].index[:num_rows_to_drop]
kepler_df = kepler_df.drop(index=rows_with_nan).reset_index(drop=True)

# Mostrar el número de filas después de eliminar
print(f"Filas restantes en kepler_df: {len(kepler_df)}")

Filas restantes en kepler_df: 5268
